# Тестовое задание

In [449]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string

In [450]:
count_vect = CountVectorizer(max_df = 0.55, min_df = 0.05, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.55, min_df = 0.05, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()

## Cкачиваем файл с данными

In [451]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [452]:
#df = pd.read_json ('reviews_Electronics_5.json', lines=True )
#df.to_csv ('Electronics_5.csv', index = None)

# Предобработка

In [453]:
dat = pd.read_csv("Electronics_5.csv")[["asin","reviewText", "overall"]]

## Добавляем бинрную оценку

In [454]:
dat["bin"] = dat.overall > 3

## Отрежем кусок даты

In [455]:
dat_t, dat = dat, dat[0:11]

## Натравим на необработанные данные

In [456]:
dat_count_deft = count_vect.fit_transform(dat.reviewText)
dat_tfidf_deft = tfidf_vect.fit_transform(dat.reviewText)


## Приводим к базовой форме

In [457]:
dat["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), dat.reviewText)))

/home/denis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Можно воспользоваться анализатором, который предоставляют векторайзеры

In [458]:
# anlz = count_vect.build_analyzer()
# dat.words = list(map(lambda s: (" ").join(anlz(s)), dat.reviewText))

## Натравим на обработанные данные

In [459]:
dat_count = count_vect.fit_transform(dat.words)
dat_tfidf = tfidf_vect.fit_transform(dat.words)

## Проверим, что получилось

In [460]:
dat

,asin,reviewText,overall,bin,words
0,0528881469,We got this GPS for my husband who is an (OTR)...,5,True,we got thi gp for my husband who is an otr ove...
1,0528881469,"I'm a professional OTR truck driver, and I bou...",1,False,profession otr truck driver and bought tnd 700...
2,0528881469,"Well, what can I say. I've had this unit in m...",3,False,well what can say ve had thi unit in my truck ...
3,0528881469,"Not going to write a long review, even thought...",2,False,not go to write long review even thought thi u...
4,0528881469,I've had mine for a year and here's what we go...,1,False,ve had mine for year and here what we got it t...
5,0594451647,I am using this with a Nook HD+. It works as d...,5,True,am use thi with nook hd it work as describ the...
6,0594451647,The cable is very wobbly and sometimes disconn...,2,False,the cabl is veri wobbl and sometim disconnect ...
7,0594451647,This adaptor is real easy to setup and use rig...,5,True,thi adaptor is real easi to setup and use righ...
8,0594451647,This adapter easily connects my Nook HD 7&#34;...,4,True,thi adapt easili connect my nook hd 34 to my h...
9,0594451647,This product really works great but I found th...,5,True,thi product realli work great but found the fo...


In [467]:
dat_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [462]:
dat_tfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02766308, 0.02766308, 0.02766308, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04362035, 0.04362035,
        0.04362035],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [468]:
dat_count_deft.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [464]:
count_vect.stop_words_

{'and',
 'as',
 'be',
 'for',
 'in',
 'is',
 'it',
 'my',
 'on',
 'that',
 'the',
 'thi',
 'to',
 'with'}